# Using LMs to generate text
Ejercicio propuesto en el capítulo 2: Transformers del libro Hands on Generative AI

In [1]:
import torch
import transformers
import numpy as np

GPU = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
CPU = torch.device("cpu")
print(GPU)

cuda:0


# Enunciado
Your goal is to fill the code in the following function. Rather than use `model​.gen⁠erate()`, the idea is to iteratively call `model()`, passing the previous tokens as input. You have to implement greedy search when `do_sample=False`, sampling when `do_sample=True`, and Top-K sampling when `do_sample=True` and top_k is not `None`. This will be a challenging task, so do not worry if you don’t come up with a solution quickly. We suggest you begin implementing greedy search and then build on top of it:

In [2]:
model = transformers.AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B")
model.to(GPU)
tokenizer = transformers.AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")

In [3]:
prompt = "Cuándo la tormenta amenazaba con caer sobre los albores del mundo"

input_tokens_ids = tokenizer(prompt,return_tensors="pt").input_ids
input_tokens_ids = input_tokens_ids.to(GPU)


In [4]:
def normalize(values : np.array):

    min_valor = np.min(values)
    valores = values + abs(min_valor)

    normalized = valores / np.sum(valores)
    return normalized


def generate(
    model, tokenizer, input_ids, max_length=50, do_sample=False, top_k=None
):
    """Generate a sequence without using model.generate()

    Args:
        model: The model to use for generation.
        tokenizer: The tokenizer to use for generation.
        input_ids: The input IDs
        max_length: The maximum length of the sequence.
        do_sample: Whether to use sampling.
        top_k: The number of tokens to sample from.
    """
    # Write your code here
    # Begin by the simplest approach, greedy decoding.
    # Then add sampling and finally top-k sampling.

    # input_ids = input_tokens_ids
    input_list = input_ids.tolist()[0]
    iterate_ids = input_ids
    iterate_list = input_list

    for _ in range(max_length):

        output_logits = model(iterate_ids).logits[0][-1].to(CPU) # Ejecuto el modelo.
                                                        # Devuelve una lista de logits
        
        if not do_sample:   # Si no está activo el do_sample, se usa simpre el más probable
            new_token_id = output_logits.argmax().tolist()
        else:

            if not top_k:   # Si no se usa top_k, se utilizan todos los posibles tokens
                np_logits = output_logits.detach().numpy()
                
                normalized_weights = normalize(np_logits)
                new_token_id = np.random.choice(len(normalized_weights),p=normalized_weights)
                
            else:           # Si se usa top_k, se ordenan los logits y se utilizan los k más probables.
                np_output_logits = output_logits.detach().numpy()
                np_args_logits = np.argsort(np_output_logits)[::-1][0:top_k]
                np_logits = np_output_logits[np_args_logits]
                
                normalized_weights = normalize(np_logits)
                new_token_id = np.random.choice(np_args_logits,p=normalized_weights)
            
            
        iterate_list.append(new_token_id)
        iterate_ids = torch.tensor([iterate_list]).to(GPU)


    output = tokenizer.decode(input_list,skip_special_tokens=True)

    return output
    

In [6]:
print(generate(model, tokenizer, input_tokens_ids,max_length=100,do_sample=True,top_k=4))

Cuándo la tormenta amenazaba con caer sobre los albores del mundo, la humanidade, como siempre ha ocurrido con las grandes tormentas del tiempo que se produjeron en la historia humana y la destruyó.
El 3 de abril 541, la Torbelline de la Tormente de 10,700 kilos de masa, que se desató sobre la tirola de Vercelli y el nacimiento de los alboles en 240 kilómetros al noreo y
